In [ ]:
!pip install transformers
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
     |████████████████████████████████| 6.6 MB 80.8 MB/s 
     |████████████████████████████████| 101 kB 15.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 74.4 MB/s 
     |████████████████████████████████| 1.1 MB 84.4 MB/s 
     |████████████████████████████████| 140 kB 76.1 MB/s 
     |████████████████████████████████| 127 kB 80.9 MB/s 
     |████████████████████████████████| 271 kB 69.0 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |█████████████

In [ ]:
from zipfile import ZipFile
from io import BytesIO
import requests
r = requests.get("http://cimalab.unal.edu.co/applications/thyroid/thyroid.zip")
print("Downloading...")
z = ZipFile(BytesIO(r.content))    
z.extractall("/content/thyroid")



Downloading...


In [ ]:
import os
import xml.etree.ElementTree as ET
from natsort import natsorted
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt


# XML and Jpeg     
def to_dataframe(path):
    dirs=natsorted(os.listdir(path))
    xml_list=[]
    img_list=[]
    for i in range(len(dirs)):
        if '.xml' in dirs[i]:
            xml_list.append(dirs[i])
        if not '.xml'  in dirs[i]:
            img_list.append(dirs[i])
    xml_list=natsorted(xml_list)
    img_list=natsorted(img_list)
    tirads=[]
    for j in range(len(xml_list)):
        tree = ET.parse(path+'/'+xml_list[j])
        a=tree.findall("./tirads")
        if a[-1].text!=None:
            case=[xml_list[j],a[-1].text]
            tirads.append(case)
    data=[]
    for k in range(len(tirads)):
        xml=tirads[k][0][:-4]
        for z in range(len(img_list)):
            if xml+'_1.jpg'==img_list[z] or xml+'_2.jpg'==img_list[z] or xml+'_3.jpg'==img_list[z]:
                m=[img_list[z],tirads[k][1]]
                data.append(m)

    df = pd.DataFrame(data,columns =['Jpeg_Name', 'Tirads'])
    return df

#Cropp Function
def croping(img,x, y, w, h):
    if abs(w)<abs(h):
        img2=np.zeros([h,h])
        img2[:,h-w:h]=img[y:y+h, x:x+w]
    if abs(h)<abs(w):  
        img2=np.zeros([w,w])
        img2[w-h:w,:]=img[y:y+h, x:x+w]
    else:
        return img
    return img2

def convert_one_channel(img):
    #if some images have 3 channels , although they are grayscale image
    if len(img.shape)>2:
        img=img[:,:,0]
        return img
    else:
        return img

#Remove Fill area from Image and Resizeing
def crop_resize(path,resize_shape):
    img=plt.imread(path)
    img=convert_one_channel(np.asarray(img))    
    kernel =( np.ones((5,5), dtype=np.float32))
    ret,thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)
    thresh = thresh.astype(np.uint8)
    a1,b1=thresh.shape
    thresh=cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel,iterations=3 )
    thresh=cv2.erode(thresh,kernel,iterations =5)
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    c_area=np.zeros([len(contours)])
    for i in range(len(contours)):
        c_area[i]= cv2.contourArea(contours[i]) 
    cnts=contours[np.argmax(c_area)]    
    x, y, w, h = cv2.boundingRect(cnts)
    roi = croping(img, x, y, w, h)
    roi=cv2.resize(roi,(resize_shape),interpolation=cv2.INTER_LANCZOS4)


    return roi


# TO Data Matrix
def to_imgmatrix(resize_shape,path,df):
    path=path+'/'  
    images=[]
    for i in range (0,len(df["Jpeg_Name"])):
        img=crop_resize(path+df["Jpeg_Name"][i],resize_shape)
            #Convert RGB to use pretrained model 
        img=np.uint8(255*(img/(np.max(img))))
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        images.append(img)
    return images

def prepare_data(path,resize_shape):
    df=to_dataframe(path)
    data=to_imgmatrix(resize_shape,path,df) 
    return df,data


def to_categoricalmatrix(df):
    #There are little categories, so i handled manually
    Y=np.zeros([len(df["Tirads"])])
    for i in range(len(df["Tirads"])):
        if df["Tirads"][i]=="2":
          Y[i]=0
        if df["Tirads"][i]=="3":
          Y[i]=0
        if df["Tirads"][i]=="4a":
          Y[i]=1
        if df["Tirads"][i]=="4b":
          Y[i]=1
        if df["Tirads"][i]=="4c":
          Y[i]=1
        if df["Tirads"][i]=="5":
          Y[i]=1
    return Y.tolist()



df,data=prepare_data("/content/thyroid",(256,256))


In [ ]:
df.head()

,Jpeg_Name,Tirads
0,2_1.jpg,2
1,3_1.jpg,4a
2,4_1.jpg,4a
3,5_1.jpg,5
4,6_1.jpg,4b


In [ ]:
y=to_categoricalmatrix(df)

In [ ]:
import os
directory = "data"
parent_dir = "/content"
path = os.path.join(parent_dir, directory)
os.makedirs(path)

for i in range(2):
  directory=str(i)
  sub_path=os.path.join(path, directory)
  os.makedirs(sub_path)

In [ ]:
for j in range(len(data)):
  label_path=str(np.int8(y[j]))
  data_path=os.path.join(path, label_path)
  cv2.imwrite(data_path+"/"+df["Jpeg_Name"][j],data[j])

In [ ]:
from datasets import load_dataset 
dataset = load_dataset("imagefolder", data_dir="/content/data")

Resolving data files:   0%|          | 0/347 [00:00<?, ?it/s]

Using custom data configuration default-c70d07ca055a5f41


Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/default-c70d07ca055a5f41/0.0.0/48efdc62d40223daee675ca093d163bcb6cb0b7d7f93eb25aebf5edca72dc597. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from PIL import Image
import torch
from torchvision import datasets, models, transforms
img_size=224
batch_size=32

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomAutocontrast(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Resize(size=(img_size,img_size))    
    ]),

    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Resize(size=(img_size,img_size))

    ]),
}



def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        data_transforms["train"](image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [data_transforms["val"](image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

splits = dataset["train"].train_test_split(test_size=0.2)
train_ds = splits['train']
val_ds = splits['test']

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

Parameter 'transform'=<function preprocess_train at 0x7f14c12a54d0> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [ ]:
from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification, TrainingArguments, Trainer
import torch
from datasets import load_metric


model_name_or_path = 'facebook/convnext-tiny-224'
metric = load_metric("f1")
labels = train_ds.features['label'].names




feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)
model = ConvNextForImageClassification.from_pretrained(model_name_or_path,    num_labels=len(labels),ignore_mismatched_sizes=True,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)})


#You can train your own model with more epoch 
args = TrainingArguments(
    "Cvt-finetuned-thyroid",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=150,
    warmup_ratio=0.1,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=50
)

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}



trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109M [00:00<?, ?B/s]

Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-tiny-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 277
  Num Epochs = 150
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 300


Epoch,Training Loss,Validation Loss,F1
0,No log,0.651789,0.847458
1,No log,0.649232,0.847458
2,No log,0.644705,0.847458
3,No log,0.638478,0.833333
4,No log,0.630739,0.852459
5,No log,0.621660,0.870968
6,No log,0.611340,0.870968
7,No log,0.599934,0.880000
8,No log,0.587955,0.888889
9,No log,0.576150,0.888889


***** Running Evaluation *****
  Num examples = 70
  Batch size = 32
Saving model checkpoint to Cvt-finetuned-thyroid/checkpoint-2
Configuration saved in Cvt-finetuned-thyroid/checkpoint-2/config.json
Model weights saved in Cvt-finetuned-thyroid/checkpoint-2/pytorch_model.bin
Feature extractor saved in Cvt-finetuned-thyroid/checkpoint-2/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 70
  Batch size = 32
Saving model checkpoint to Cvt-finetuned-thyroid/checkpoint-4
Configuration saved in Cvt-finetuned-thyroid/checkpoint-4/config.json
Model weights saved in Cvt-finetuned-thyroid/checkpoint-4/pytorch_model.bin
Feature extractor saved in Cvt-finetuned-thyroid/checkpoint-4/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 70
  Batch size = 32
Saving model checkpoint to Cvt-finetuned-thyroid/checkpoint-6
Configuration saved in Cvt-finetuned-thyroid/checkpoint-6/config.json
Model weights saved in Cvt-finetuned-thyroid/checkpoint-6/pytorch_model

***** train metrics *****
  epoch                    =      149.89
  total_flos               = 971859038GF
  train_loss               =       0.368
  train_runtime            =  0:14:37.20
  train_samples_per_second =      47.366
  train_steps_per_second   =       0.342


In [ ]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 70
  Batch size = 32


***** eval metrics *****
  epoch                   =     149.89
  eval_f1                 =     0.9032
  eval_loss               =     0.4506
  eval_runtime            = 0:00:00.57
  eval_samples_per_second =    122.228
  eval_steps_per_second   =      5.238


In [ ]:
!pip install huggingface_hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login 
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from huggingface_hub import Repository
repo = Repository( local_dir = 'SerdarHelli', clone_from='SerdarHelli/ThyroidTumorClassification' )


/content/SerdarHelli is already a clone of https://huggingface.co/SerdarHelli/ThyroidTumorClassification. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
%cp   /content/Cvt-finetuned-thyroid/all_results.json /content/SerdarHelli/all_results.json
%cp   /content/Cvt-finetuned-thyroid/eval_results.json /content/SerdarHelli/eval_results.json
%cp   /content/Cvt-finetuned-thyroid/train_results.json /content/SerdarHelli/train_results.json 
%cp   /content/Cvt-finetuned-thyroid/trainer_state.json /content/SerdarHelli/trainer_state.json
%cp   /content/Cvt-finetuned-thyroid/training_args.bin /content/SerdarHelli/training_args.bin


In [ ]:
trainer.save_model("/content/SerdarHelli")

repo.push_to_hub("ThyroidTumorClassification")

Saving model checkpoint to /content/SerdarHelli
Configuration saved in /content/SerdarHelli/config.json
Model weights saved in /content/SerdarHelli/pytorch_model.bin
Feature extractor saved in /content/SerdarHelli/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 3.34k/106M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.23k/3.23k [00:00<?, ?B/s]

To https://huggingface.co/SerdarHelli/ThyroidTumorClassification
   9e9423c..2a3559e  main -> main

   9e9423c..2a3559e  main -> main



'https://huggingface.co/SerdarHelli/ThyroidTumorClassification/commit/2a3559ecdb37289bcd279b9ee8e16d1e5a6fad48'